<a href="https://colab.research.google.com/github/Bhuto1998/Online-Cluster-Validity-Index/blob/main/Batch_Online_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algorithm to Update Sample Covariance Matrix:
Let Sample covariance matrix be $S_{n} = \frac{1}{n-1} \sum_{i=1}^{n}(v_{i} - m_{n})(v_{i} - m_{n})^{T}$ where $v_{i}$'s are the datapoints and $m_{n}$ is the mean. Now let $m_{t}$ is the mean vector at time $t$. Our task is to determine the value of $S_{t+1}$ using $S_{t}$ and some key statistic. \\

 $S_{t+1} = \frac{1}{t} \sum_{i=1}^{t+1}(v_{i} - m_{t+1})(v_{i} - m_{t+1})^{T} = \frac{1}{t} \left[\sum_{i=1}^{t}(v_{i} - m_{t+1})(v_{i} - m_{t+1})^{T} + (v_{t+1} - m_{t+1})(v_{t+1}-m_{t+1})^{T} \right]$
 Let , $\delta_{t+1} = (v_{t+1} - m_{t+1})(v_{t+1}-m_{t+1})^{T}$ and $M_{t+1} = (m_{t}-m_{t+1})(m_{t}-m_{t+1})^{T}$ \\
 Now, $(v_{i}-m_{t+1})(v_{i}-m_{t+1})^{T} = (v_{i}-m_{t}+m_{t}-m_{t+1})(v_{i}-m_{t}+m_{t}-m_{t+1})^{T} \\= (v_{i}-m_{t})(v_{i}-m_{t})^{T} + (v_{i}-m_{t})(m_{t}-m_{t+1})^{T} + (m_{t}-m_{t+1})(v_{i}-m_{t})^{T} + (m_{t}-m_{t+1})(m_{t}-m_{t+1})^{T}$
 Notice that, $\sum_{i=1}^{t}(v_{i}-m_{t})(m_{t}-m_{t+1})^{T} = \sum_{i=1}^{t} (m_{t}-m_{t+1})(v_{i}-m_{t})^{T} = 0$ \\
 Hence, $S_{t+1} = \frac{1}{t}\left[ (t-1)S_{t} + tM_{t+1} + \delta_{t+1}  \right]$ 

 **Algorithm**
 1. Start from $S_{2} , m_{2}$
 2. For each t>2 and new data point $v_{t}$:
  * $M_{t} = (m_{t-1}-m_{t})(m_{t-1}-m_{t})^{T}$
  * $\delta_{t} = (v_{t} - m_{})(v_{t}-m_{t})^{T}$ 
  * $S_{t} = \frac{1}{t-1}\left[ (t-2)S_{t-1} + (t-1)M_{t} + \delta_{t}  \right
  ]$ 



# Batch-Online Clustering Algorithm Using Cluster Validity Index:

update(a,S) function takes a new point a and sample covariance S as input and returns the updated sample covariance. ocvi(a) function takes a new point as input and add it to the existing data points and returns the optimal cluster number. $f$ is a fixed value which is the number of samples required to call a cluster "stable".

1. For First $N$ points (typically 1000 or 2000): Find the $\mathcal{I}^{*}$ value. Let it be $K$.
2. Perform exponential fading SK-mean clustering with $\lambda = 0.9$ and evaluate the following:
 * centroids/mean points of each cluster: $C_{1},...,C_{K}$
 * counters (number of points): $n_{1},...,n_{K}$
 * Covariance matrices: $S_{1},....,S_{K}$
 * Maximum mohalanobis distance of two points in one cluster: $maxmoha = [m_{1},...,m_{K}]$

3. For each new point $x_{new}$ Perform the following: 
 * Find out the cluster which has the minimum mohalanobis distance from this point. Let it be cluster $r$ and let that distance be $d$.
 *  If $d < maxmoha[r]$:
    * $ p = unif(0,1)$
    * If $ p > \frac{d}{maxmoha[r]}$ or $ n_{r} < f: S_{r} = update(x_{new} , S_{r})$
  * else:
    * $t = ocvi(x_{new})$
    * If $t==K: S_{r} = update(x_{new},S_{r})$
    * Else: $C_{K+1} = x_{new}$ and $S_{K+1} = I$
  * $C_{r} = \frac{n_{r}C_{r} + x_{new}}{n_{r} + 1}$ and $n_{r} = n_{r} + 1$



# Reasoning:
In the first we wait to get a stable structure of the dataset. Note that we are using mohalanobis distance instead of regular euclidean distance to capture the deviation in terms of the distribution of the cluster and not just the cluster center. Now if the mohalanobis distance of the new point from the predicted cluster is smaller than the maximum mohalanobis distance that means it is highly likely the new point indeed belongs to the cluster and we have already seen points similar to that one. So we use it to update sample covariance probabilistically. Note that if the number of points observed for a cluster is less than a particular threshold ($f$) we say that the cluster is not yet stable (i.e. the sample covariance is not a good estimate of population covariance). Typically value of such $f$ depends on the dimension of the dataset. For univariate 35 points is good enough but for bivariate we might need more than 50 points. (This is based on large sample asymptotics of the sample covariance). In the case that the distance is larger than the maximum mohalanobis distance itself, we have a potential suspect for a new cluster itself. We run the ocvi function to check if the optimal number of cluster changed. If it did we add a new cluster and in the othercase we use that point to update the Sample covariance of the predicted cluster.  